# Задание №1

Есть Pandas DataFrame со столбцами [“customer_id”, “product_id”, “timestamp”], 
который содержит данные по просмотрам товаров на сайте. 
Есть проблема – просмотры одного customer_id не разбиты на сессии (появления на сайте). 
Мы хотим разместить сессии так, чтобы сессией считались все смежные просмотры, между которыми не более 3 минут.

Написать методом который создаст в Pandas DataFrame столбец session_id и проставит в нем уникальный int id для каждой сессии.

У каждого пользователя может быть по несколько сессий. Исходный DataFrame может быть большим – до 100 млн строк.

## Подготовка входных данных

In [1]:
import random
from datetime import datetime, timedelta
from typing import List

import pandas as pd
from tqdm import tqdm

pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)

random.seed(0)

In [2]:
SAMPLE_SIZE = 10_000_000 # Тут можно указать размер выборки

In [3]:
def get_dates(SAMPLE_SIZE: int):
    curr_time = datetime.now().replace(microsecond=0)

    date_list = [] # Создаем список из 'появлений' пользователя на сайте
    for _ in range(SAMPLE_SIZE):
        curr_time = curr_time + timedelta(seconds=random.randrange(10, 40)) # Каждый новый коннект раз в 10-40 секунд
        date_list.append(curr_time)
        
    return date_list

In [4]:
def create_dataset(SAMPLE_SIZE: int, date_list: List):
    customers = [random.randrange(4) for user in range(SAMPLE_SIZE)] # Создаем список из уникальных id клиентов которые заходили. (Взял небольшое количество чтобы легче было проверить)
    product_id = [random.randrange(100000, 10000000) for user in range(SAMPLE_SIZE)] # Список уникальных продуктов, которые смотрели клиенты.
    timestamp = [t.timestamp() for t in date_list] # Переводим datetime даты в timestamp, как в условии задачи.

    df = pd.DataFrame({
        'customer_id': customers,
        'product_id': product_id,
        'timestamp': timestamp
    })
    
    return df

In [5]:
date_list = get_dates(SAMPLE_SIZE)
df = create_dataset(SAMPLE_SIZE, date_list)

Исходные данные:

In [6]:
df.head()

,customer_id,product_id,timestamp
0,0,1445238,1.667065e+09
1,3,2642372,1.667065e+09
2,3,9347494,1.667065e+09
3,0,7287409,1.667065e+09
4,0,4423863,1.667065e+09


## Решение

In [7]:
# Создадим словарь, формата {client_id: [sess_id, last_time_viewed]}

def create_customers_dict(df: pd.DataFrame):
    dict_init_df = df.drop_duplicates(subset='customer_id', keep='first')[['customer_id', 'timestamp']]
    dict_init_df = dict_init_df.to_dict('records')

    customers_dict = {}
    for customer_init in dict_init_df:
        customer_id, last_time_viewed = tuple(customer_init.values())
        sess_id = None # По началу у всех клиентов id сессии = None
        customers_dict[customer_id] = [sess_id, last_time_viewed]

    return customers_dict

In [8]:
# Функция которая создает в Pandas DataFrame столбец session_id и проставляет в нем уникальный int id для каждой сессии.

def define_sessions(df: pd.DataFrame, 
                    customers_dict: dict, 
                    num_of_sessions=None):
    
    def get_session(customer_id, timestamp): # Вложенная функция, узнает сессию для каждой строки
        nonlocal num_of_sessions
        
        # Вычисляем время между соседними активностями(просмотрами) одного поьзователя
        customer_last_viewed_time = datetime.fromtimestamp(customers_dict[customer_id][1])
        customer_new_viewed_time = datetime.fromtimestamp(timestamp)
        delta_minutes = (customer_new_viewed_time - customer_last_viewed_time).seconds // 60
        
        # Если сессий еще не было определено, создаем ее, 
        # записываем в текущую pd строку номер этой сессии и также обновляем информацию о последнем подключении в словаре клиента
        if num_of_sessions == None:  
            num_of_sessions = 0
            session_id = num_of_sessions
            customers_dict[customer_id][0] = num_of_sessions
        else: # Если сессии уже есть, то смотрим, существуют ли они у клиента.
            # Если нет, то сессия клиента будет равна числу существующих сессий + 1. Обновляем информацию в словарике
            if customers_dict[customer_id][0] == None:
                num_of_sessions += 1
                session_id = num_of_sessions
                customers_dict[customer_id][0] = num_of_sessions
            # Если у клиента тоже есть/была сессия, то смотрим как давно она была.
            else:
                # Если прошло <= 3 минут - то записываем старую сессию, так как по условию она еще активна.
                if delta_minutes <= 3:
                    session_id = customers_dict[customer_id][0]
                # Если прошло > 3 минут - то записываем сессию как новую.
                else:
                    num_of_sessions += 1
                    session_id = num_of_sessions
        # В словаре клиента обновляем информацию о последней активности
        customers_dict[customer_id][1] = customer_new_viewed_time.timestamp()
        
        return session_id
    
    return list(map(get_session, tqdm(df['customer_id']), df['timestamp']))

## Результат

In [9]:
customers_dict = create_customers_dict(df)

In [10]:
%time df['session_id'] = define_sessions(df, customers_dict)

100%|████████████████████████████████████████████████████████████████████████████████████| 10000000/10000000 [00:40<00:00, 246967.01it/s]


CPU times: total: 43.9 s
Wall time: 44.2 s


In [11]:
df.head()

,customer_id,product_id,timestamp,session_id
0,0,1445238,1.667065e+09,0
1,3,2642372,1.667065e+09,1
2,3,9347494,1.667065e+09,1
3,0,7287409,1.667065e+09,0
4,0,4423863,1.667065e+09,0
